In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, Dataset

# Dataset Class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.seq_len]
        return torch.tensor(x, dtype=torch.float32)

# Transformer-based Model
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, ff_dim, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.input_projection = nn.Linear(input_dim, model_dim)
        self.positional_encoding = nn.Parameter(torch.randn(1, 1000, model_dim))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim, 
            nhead=num_heads, 
            dim_feedforward=ff_dim, 
            dropout=dropout
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_projection = nn.Linear(model_dim, input_dim)

    def forward(self, x):
        seq_len = x.size(2)
        x = self.input_projection(x) + self.positional_encoding[:, :seq_len, :]
        x = self.transformer(x)
        return self.output_projection(x)

# Noise Scheduler
class NoiseScheduler:
    def __init__(self, num_steps, beta_start=0.0001, beta_end=0.02):
        self.num_steps = num_steps
        self.betas = torch.linspace(beta_start, beta_end, num_steps)
        self.alphas = 1.0 - self.betas
        self.alpha_bar = torch.cumprod(self.alphas, dim=0)

    def add_noise(self, x, step):
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[step])
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar[step])
        noise = torch.randn_like(x)
        return sqrt_alpha_bar * x + sqrt_one_minus_alpha_bar * noise, noise

# Training Function
def train_model(model, dataset, scheduler, num_steps=1000, epochs=10, batch_size=64, lr=1e-4):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            step = torch.randint(0, num_steps, (1,)).item()
            noisy_x, noise = scheduler.add_noise(batch, step)
            # print(noisy_x.unsqueeze(0).shape)
            pred_noise = model(noisy_x).squeeze(0)
            loss = mse_loss(pred_noise, noise)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}")

# Synthesis Function
def generate_series(model, scheduler, seq_len, num_steps):
    model.eval()
    with torch.no_grad():
        x = torch.randn(1, seq_len, model.output_projection.out_features)
        print(x)
        for step in reversed(range(num_steps)):
            pred_noise = model(x)
            print(pred_noise)
            alpha_bar = scheduler.alpha_bar[step]
            if step > 0:
                noise = torch.randn_like(x)
                x = (
                    (x - pred_noise * (1 - alpha_bar).sqrt()) / alpha_bar.sqrt() 
                    + noise * scheduler.betas[step].sqrt()
                )
            else:
                x = (x - pred_noise * (1 - alpha_bar).sqrt()) / alpha_bar.sqrt()
        return x.squeeze(0)

# Example Usage
if __name__ == "__main__":
    # Simulated data
    data = np.sin(np.linspace(0, 100, 10000)).reshape(-1, 1)
    seq_len = 100
    dataset = TimeSeriesDataset(data, seq_len)

    # Model and scheduler
    model = TimeSeriesTransformer(
        input_dim=1, model_dim=64, num_heads=4, num_layers=4, ff_dim=256
    )
    scheduler = NoiseScheduler(num_steps=1000)

    # Training
    train_model(model, dataset, scheduler, num_steps=1000, epochs=50)

    # Generation
    generated_series = generate_series(model, scheduler, seq_len=100, num_steps=1000)
    print(generated_series)


Epoch 1, Loss: 0.33614987730979917
Epoch 2, Loss: 0.2473671131436863
Epoch 3, Loss: 0.28617843569767093
Epoch 4, Loss: 0.2485051393869423
Epoch 5, Loss: 0.2742529645682343
Epoch 6, Loss: 0.22172953818113572
Epoch 7, Loss: 0.24056071995366965
Epoch 8, Loss: 0.1944707153545272
Epoch 9, Loss: 0.24386910817796184
Epoch 10, Loss: 0.21020692674022529
Epoch 11, Loss: 0.2628952405625774
Epoch 12, Loss: 0.17444221253596967
Epoch 13, Loss: 0.24292712558601653
Epoch 14, Loss: 0.24374597808045725
Epoch 15, Loss: 0.20824644119749147
Epoch 16, Loss: 0.21578752465786472
Epoch 17, Loss: 0.19446923021046866
Epoch 18, Loss: 0.2273079549082585
Epoch 19, Loss: 0.19545744445715701
Epoch 20, Loss: 0.1963582293070372
Epoch 21, Loss: 0.20825113092338846
Epoch 22, Loss: 0.20723004219753127
Epoch 23, Loss: 0.22999226427787253
Epoch 24, Loss: 0.2230760783317589
Epoch 25, Loss: 0.17704695928541403
Epoch 26, Loss: 0.20154188974430004
Epoch 27, Loss: 0.16456178614809627
Epoch 28, Loss: 0.22244491464488447
Epoch 29,